# Vertex list generation

We want to save our field data and use it to generate a 3-d model in blender.  Current idea is to make polygonal mesh using the python-blender interface.  In order to do this, need to generate a  vertex list (and later a face list possibly, not sure if that is automated in blender).  Here we try to do that without using for loops.



In [1]:
import numpy as np

### Toy example

In [2]:
Nx=5
Nz=7
xs, zs  = np.linspace(-2,2,Nx), np.linspace(-4,4,Nz)

In [3]:
X, Z = np.meshgrid(xs, zs)
X.shape
# X, Z

(7, 5)

What order does the below put x,z coordinates into? (man terrible grammar)

In [4]:
I = np.array([X,Z, np.zeros_like(Z)])

In [5]:
Verts = I.T.reshape((Nx*Nz,3))
# Verts

Lexicographic in x coordinate.  Indices below mimic this:

In [6]:
inds = np.arange(0, Nx*Nz).reshape((Nx, Nz)).T

In [7]:
inds  # shape is same as X, Z in meshgrid, but entries give index in later lexicographic ordering.

array([[ 0,  7, 14, 21, 28],
       [ 1,  8, 15, 22, 29],
       [ 2,  9, 16, 23, 30],
       [ 3, 10, 17, 24, 31],
       [ 4, 11, 18, 25, 32],
       [ 5, 12, 19, 26, 33],
       [ 6, 13, 20, 27, 34]])

We'll make face index lists using the upper left coordinates as the bases.  Let's get those:

In [8]:
inds.shape

(7, 5)

In [9]:
bases = inds[:Nz-1, :Nx-1]  # right, that's easy, just a slice  (up to len(zs)-1, len(xs)-1 as we transposed)
fbases = bases.T.flatten()
bases, fbases

(array([[ 0,  7, 14, 21],
        [ 1,  8, 15, 22],
        [ 2,  9, 16, 23],
        [ 3, 10, 17, 24],
        [ 4, 11, 18, 25],
        [ 5, 12, 19, 26]]),
 array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17, 18,
        19, 21, 22, 23, 24, 25, 26]))

To make face index data, could loop over this base list.  Looks like the list will be the index at hand, say 0 at first, followed by that index plus len(z), here that gives 4, then that number +1, 5, then the original index plus one.  Let's try that:

In [10]:
test_faces = np.array([[i, i+Nz, i+Nz+1, i+1] for i in fbases])

In [12]:
def faces(xs, zs=None):
    '''Get face index list for blender meshing from domain mesh data.'''
    if len(xs.shape) > 1:  # assume meshgrid data provided.
        Nz, Nx = xs.shape[0], xs.shape[1]  # Dimension swap? Check
        l_inds = Nx * Nz
    elif len(xs.shape) == 1 and zs is not None:
        Nx, Nz= len(xs), len(zs)
        l_inds = Nx * Nz
    else:
        raise ValueError('Must provide either meshgrid (2d array) or two 1d arrays xs and zs as keyword argument zs=.')
    indices =  np.arange(0, l_inds).reshape((Nx, Nz)).T  # Note transpose
    fbases = indices[:Nz-1, :Nx-1].T.flatten()  # Note transpose and index switch due to transpose
    faces = np.array([[i, i+Nz, i+Nz+1, i+1] for i in fbases], dtype=int)
    return faces
    

In [13]:
test_faces, faces(X)

(array([[ 0,  7,  8,  1],
        [ 1,  8,  9,  2],
        [ 2,  9, 10,  3],
        [ 3, 10, 11,  4],
        [ 4, 11, 12,  5],
        [ 5, 12, 13,  6],
        [ 7, 14, 15,  8],
        [ 8, 15, 16,  9],
        [ 9, 16, 17, 10],
        [10, 17, 18, 11],
        [11, 18, 19, 12],
        [12, 19, 20, 13],
        [14, 21, 22, 15],
        [15, 22, 23, 16],
        [16, 23, 24, 17],
        [17, 24, 25, 18],
        [18, 25, 26, 19],
        [19, 26, 27, 20],
        [21, 28, 29, 22],
        [22, 29, 30, 23],
        [23, 30, 31, 24],
        [24, 31, 32, 25],
        [25, 32, 33, 26],
        [26, 33, 34, 27]]),
 array([[ 0,  7,  8,  1],
        [ 1,  8,  9,  2],
        [ 2,  9, 10,  3],
        [ 3, 10, 11,  4],
        [ 4, 11, 12,  5],
        [ 5, 12, 13,  6],
        [ 7, 14, 15,  8],
        [ 8, 15, 16,  9],
        [ 9, 16, 17, 10],
        [10, 17, 18, 11],
        [11, 18, 19, 12],
        [12, 19, 20, 13],
        [14, 21, 22, 15],
        [15, 22, 23, 16],
        [1

In [14]:
# inds, test_faces

Looking good! We actually can probably skip the creation of the 2d index array by skipping in arange.  Later though.

In [331]:
np.save('/home/pv/test_faces', test_faces), np.save('/home/pv/test_vertices', Verts)

(None, None)

### Adding field data

Actually, this shouldn't really change anything.

In [401]:
Fs = np.cos(X**2 + Z**2)
# Fs

In [402]:
D = np.array([X, Z, Fs])
# D, inds

In [403]:
verts = D.T.reshape((Nx*Nz,3))
# verts, inds.flatten()

Yeah, face index list is the same.

In [319]:
np.save('/home/pv/test_faces', test_faces), np.save('/home/pv/test_vertices', verts)

(None, None)

### Functions

In [424]:
def faces(xs, zs=None):
    '''Get face index list for blender meshing from domain mesh data.'''
    if len(xs.shape) > 1:  # assume meshgrid data provided.
        Nz, Nx = xs.shape[0], xs.shape[1]  # Dimension swap? Check
        l_inds = Nx * Nz
    elif len(xs.shape) == 1 and zs is not None:
        Nx, Nz= len(xs), len(zs)
        l_inds = Nx * Nz
    else:
        raise ValueError('Must provide either meshgrid (2d array) or two 1d arrays xs and zs as keyword argument zs=.')
    indices =  np.arange(0, l_inds).reshape((Nx, Nz)).T  # Note transpose
    fbases = indices[:Nz-1, :Nx-1].T.flatten()  # Note transpose and index switch due to transpose
    faces = np.array([[i, i+Nz, i+Nz+1, i+1] for i in fbases], dtype=int)
    return faces
    

In [425]:
faces(xs, zs=zs)

array([[ 0,  7,  8,  1],
       [ 1,  8,  9,  2],
       [ 2,  9, 10,  3],
       [ 3, 10, 11,  4],
       [ 4, 11, 12,  5],
       [ 5, 12, 13,  6],
       [ 7, 14, 15,  8],
       [ 8, 15, 16,  9],
       [ 9, 16, 17, 10],
       [10, 17, 18, 11],
       [11, 18, 19, 12],
       [12, 19, 20, 13],
       [14, 21, 22, 15],
       [15, 22, 23, 16],
       [16, 23, 24, 17],
       [17, 24, 25, 18],
       [18, 25, 26, 19],
       [19, 26, 27, 20],
       [21, 28, 29, 22],
       [22, 29, 30, 23],
       [23, 30, 31, 24],
       [24, 31, 32, 25],
       [25, 32, 33, 26],
       [26, 33, 34, 27]])

### Actual data

In [426]:
ys = np.load('rad_field.npy')

In [427]:
ys.real

array([[-1.75692326e-06, -2.64074919e-06, -5.18284357e-06, ...,
        -1.39062390e-04, -1.36759021e-04, -1.36113920e-04],
       [-2.43072236e-05, -2.14869654e-05, -1.82335413e-05, ...,
        -1.48667483e-04, -1.52153489e-04, -1.55206328e-04],
       [-4.53495696e-05, -3.97126291e-05, -3.32004007e-05, ...,
        -1.53474886e-04, -1.60201580e-04, -1.66052986e-04],
       ...,
       [-1.13314406e-02, -1.19249455e-02, -1.23986354e-02, ...,
        -1.23894230e-02, -1.19157167e-02, -1.13221953e-02],
       [-1.06792328e-02, -1.14049564e-02, -1.20139149e-02, ...,
        -1.19748330e-02, -1.13658048e-02, -1.06400115e-02],
       [-9.72032508e-03, -1.05598343e-02, -1.12886941e-02, ...,
        -1.12217544e-02, -1.04927751e-02, -9.65314658e-03]])

In [428]:
Xs, Zs = np.load('radX.npy'), np.load('radZ.npy')
Data = np.array([Xs, Zs, 5*ys.real], dtype=float)


In [429]:
L = Xs.shape[0] * Xs.shape[1]
vertices = Data.T.reshape((L,3))
vertices

array([[-2.00000000e+01,  0.00000000e+00, -8.78461629e-06],
       [-2.00000000e+01,  5.01043841e-02, -1.21536118e-04],
       [-2.00000000e+01,  1.00208768e-01, -2.26747848e-04],
       ...,
       [ 2.00000000e+01,  2.38997912e+01, -5.66109764e-02],
       [ 2.00000000e+01,  2.39498956e+01, -5.32000575e-02],
       [ 2.00000000e+01,  2.40000000e+01, -4.82657329e-02]])

In [430]:
face_indices = faces(Xs)

In [431]:
face_indices.shape

(574321, 4)

In [432]:
np.max(face_indices)

575999

In [433]:
import os

In [434]:
os.getcwd()

'/home/pv/local/slab/notebooks/debug'

In [435]:
np.save('/home/pv/face_indices', face_indices), np.save('/home/pv/vertices', vertices)

(None, None)